In [1]:
from typing import Any
from torchvision import transforms
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage
import cv2
from anomalib import TaskType
from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import Padim
import os
from pathlib import Path

In [2]:
output_path = "D:/Shravtek/neutech_airfillter/anamolib/Models/bottle/epoch_3_works_good"

print(output_path)
openvino_model_path = output_path+"/weights"+"/openvino"+"/model.bin"
metadata = output_path+"/weights"+"/openvino"+"/metadata.json"
# print(openvino_model_path.exists(), metadata.exists())
inferencer = OpenVINOInferencer(
    path=openvino_model_path,  # Path to the OpenVINO IR model.
    metadata=metadata,  # Path to the metadata file.
    device="CPU",  # We would like to run it on an Intel CPU.
)

D:/Shravtek/neutech_airfillter/anamolib/Models/bottle/epoch_3_works_good


In [7]:
image_path = "D:/Shravtek/neutech_airfillter/anamolib/anomalib/datasets/bottle/bad/011.png"
#good/008.png
#good/009.png
#bad/020.png
#bad/011.png
image = read_image(path=image_path)
predictions = inferencer.predict(image=image_path)
print(predictions.pred_score, predictions.pred_label)

print(type(predictions.pred_label))
if predictions.pred_label=="ABNORMAL":
    print("ABNORMAL")
    cv2.putText(image,str("NOK"), (100, 100), cv2.FONT_HERSHEY_SIMPLEX,2,(0, 0, 255), 3)
else:
    print("NORMAL")
    cv2.putText(image,str("OK"), (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2,(0, 255, 0), 3)

cv2.imshow("image",cv2.resize(image,(640,640)))
# cv2.imshow("predictions.pred_mask",cv2.resize(predictions.pred_mask,(640,640)))
cv2.imshow("Anomolies",cv2.resize(predictions.segmentations,(640,640)))

cv2.waitKey(0)
cv2.destroyAllWindows()

0.7648072810710542 ABNORMAL
<class 'str'>
ABNORMAL


In [3]:
import os
import cv2
import csv

# Function to perform inference on an image
def perform_inference(image_path, inferencer):
    image = cv2.imread(image_path)
    predictions = inferencer.predict(image=image_path)
    if predictions.pred_label == "ABNORMAL":
        result = -1
    else:
        result = 1
    return result

In [9]:


# Path to the folder containing images
folder_path = "D:/Shravtek/neutech_airfillter/anamolib/anomalib/datasets/bottle/test/contamination"

# Initialize the list to store results
results = []

# Iterate over each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".png"):
        image_path = os.path.join(folder_path, filename)
        result = perform_inference(image_path, inferencer)
        results.append((filename, result))

# Path to save the CSV file
csv_file_path = "results.csv"

# Write results to CSV file
with open(csv_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image', 'Result'])
    for result in results:
        writer.writerow(result)

print("Results saved to:", csv_file_path)


Results saved to: results.csv
